# Text processing in pandas

Text processing functionality in pandas is located under the `Series.str` namespace.

[Series.str.split](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html) splits each string in a Series on a given separator and returns a Series of lists.

## Create demo data

We leverage the [io.StringIO](https://docs.python.org/3/library/io.html#io.StringIO) class from Python's standard library to create demo data in CSV format from an inline string.

In [1]:
import io
import pandas as pd

In [2]:
csv = io.StringIO('\n'.join([
    'id,name,phones',
    '1,Russel,"01234567,02345678,03456789"',
    '2,Patrick,"04123456,05234567,06345678"',
]))

print(csv.getvalue())

id,name,phones
1,Russel,"01234567,02345678,03456789"
2,Patrick,"04123456,05234567,06345678"


## Read data into dataframe

We provide `csv` as file-like input to [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html).

We specify that the `id` column should populate the DataFrame's index.

Additionally, we explicitely specify `string` datatype for `name` and `phones` columns.

In [3]:
df = pd.read_csv(
    csv,
    index_col='id',
    dtype={
        'name': 'string',
        'phones': 'string',
    },
)
df

,name,phones
id,,
1,Russel,"01234567,02345678,03456789"
2,Patrick,"04123456,05234567,06345678"


## Split into rows

Entries in the `phones` column contain multiple phone numbers separated by ','.
To make individual phone numbers accessible for further processing and analysis we have to reshape the DataFrame to hold unique rows for each phone number.
We can achieve this in a two-step process:

1.   Split each entry in the `phones` column into a list of strings using `Series.str.split`.
2.   Distribute the list elements to individual rows using `Series.explode`.

[Series.explode](https://pandas.pydata.org/docs/reference/api/pandas.Series.explode.html) transforms list items in a Series of lists (or list-like objects) into separate entries with index labels replicated.

When we perform these steps on the `phones` column we obtain a Series of individual phone numbers with index labels carried over from source entries.

In [4]:
df.phones.str.split(',').explode().astype('string')

id
1    01234567
1    02345678
1    03456789
2    04123456
2    05234567
2    06345678
Name: phones, dtype: string

At this point we can join the resulting Series to our initial DataFrame. Instead, we can arrive at the same result in a single expression.

In [5]:
df.assign(
    phones=df.phones.str.split(',')
).explode('phones').astype({'phones': 'string'})

,name,phones
id,,
1,Russel,01234567
1,Russel,02345678
1,Russel,03456789
2,Patrick,04123456
2,Patrick,05234567
2,Patrick,06345678


First, we use [DataFrame.assign](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html) to overwrite the initial `phones` column with the Series returned from the split operation.

Then we call `explode('phones')` on the resulting DataFrame.
[DataFrame.explode](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) works like `Series.explode` yet expects a column to operate on. Lastly, we change the datatype of the new `phones` column to `string`.

After this operation the index `id` is no longer unique.

Next, we are going to look at the case where the `phones` column contains additional information about the type of a phone number.

## Split into columns

In [6]:
csv = io.StringIO('\n'.join([
    'id,name,phones',
    '1,Russel,"mobil: 01234567,work: 02345678"',
    '2,Patrick,"mobil: 04123456,private: 05234567"',
]))

df = pd.read_csv(
    csv,
    index_col='id',
    dtype={'name': 'string', 'phones': 'string'},
)
df

,name,phones
id,,
1,Russel,"mobil: 01234567,work: 02345678"
2,Patrick,"mobil: 04123456,private: 05234567"


As in the previous case we split on ',' and explode to individual entries.

In [7]:
df.phones.str.split(',').explode().astype('string')

id
1      mobil: 01234567
1       work: 02345678
2      mobil: 04123456
2    private: 05234567
Name: phones, dtype: string

We need a second split operation, this time on the ':[SPACE]' character combination.

In contrast to the previous split we like the resulting fragments expanded into separate columns in the resulting DataFrame. 

In [8]:
(
    df.phones
    .str.split(',').explode()
    .str.split(': ', expand=True)
    .rename(columns={
        0: 'phone_type',
        1: 'phone_number',
    })
)

,phone_type,phone_number
id,,
1,mobil,01234567
1,work,02345678
2,mobil,04123456
2,private,05234567


When using `expand=True`, the split elements are expanded into separate columns. If `NaN` is present, it is propagated throughout the columns during the split.

So far, we have transformed the `phones` Series into a DataFrame with columns `phone_type` and `phone_number`.

In the following code snippet we combine the processing of the `phones` column with a join to the initial DataFrame in one expression.
Lastly, we drop the then redundant `phones` column from the resulting DataFrame.

In [9]:
df.join(
    df.phones
    .str.split(',').explode()
    .str.split(': ', expand=True)
    .rename(columns={
        0: 'phone_type',
        1: 'phone_number',
    })
    .astype({'phone_type': 'category'})
).drop('phones', axis='columns')

,name,phone_type,phone_number
id,,,
1,Russel,mobil,01234567
1,Russel,work,02345678
2,Patrick,mobil,04123456
2,Patrick,private,05234567


## Reshaping to wide format

We can reshape the resulting DataFrame to wide format using the following idiom:

```python
DataFrame.set_index('<COLUMN>', append=True).unstack('<COLUMN>')
```

[DataFrame.set_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html) sets the DataFrame index from existing columns.
By specifying `append=True` the column is added to the existing index instead of replacing it.
Thus, the returned DataFrame contains a hierarchical index composed of the initial index extended by the specified column.

[DataFrame.unstack](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html) moves a level from the hierarchical (row) index into a level of the column index.

In [10]:
(
    df.phones
    .str.split(',').explode()
    .str.split(': ', expand=True)
    .rename(columns={
        0: 'phone_type',
        1: 'phone_number',
    })
    .set_index('phone_type', append=True)
    .unstack('phone_type')
    .droplevel(0, axis='columns')
)

phone_type,mobil,private,work
id,,,
1,01234567,NaN,02345678
2,04123456,05234567,NaN


Now, we can join the reshaped DataFrame to the main data and drop the now redundant `phones` column.

In [11]:
df.join(
    df.phones
    .str.split(',').explode().astype('string')
    .str.split(': ', expand=True)
    .rename(columns={
        0: 'phone_type',
        1: 'phone_number',
    })
    .set_index(['phone_type'], append=True)
    .unstack('phone_type').droplevel(0, axis='columns')
).drop('phones', axis='columns')

,name,mobil,private,work
id,,,,
1,Russel,01234567,<NA>,02345678
2,Patrick,04123456,05234567,<NA>


## Data cleaning using regular expressions

We can arrive at the same result as in the previous case with fewer lines of code by leveraging regular expressions in combination with [Series.str.extract](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.extract.html).

In [12]:
(
    df.phones
    .str.split(',').explode().astype('string')
    .str.extract(r'^(?P<phone_type>\w+):\s(?P<phone_number>\d+$)')
)

,phone_type,phone_number
id,,
1,mobil,01234567
1,work,02345678
2,mobil,04123456
2,private,05234567
